In [1]:
from torchinfo import summary
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
import nltk
import pandas as pd
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import csv
from PIL import Image
from caption_model.CNN_RNN import EncoderCNN, DecoderRNN
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu

# Device configurationresul
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')

In [2]:
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':64,
        'epochs':10000,
        'data_path':'../../data/dataset/coco/',
        'train_csv':'train_annotation.csv',
        'val_csv':'val_annotation.csv',
        'vocab_path':'../../data/dataset/coco/vocab.pkl',
        'embed_size':300,
        'hidden_size':512,
        'num_layers':1,}

In [3]:
class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self, data_path, csv, class_dataset, vocab, transform=None):
        """Set the path for images, captions and vocabulary wrapper.
        
        Args:
            root: image directory.
            json: coco annotation file path.
            vocab: vocabulary wrapper.
            transform: image transformer.
        """
        self.root = data_path+class_dataset+'/'
        self.df = pd.read_csv(data_path+csv)
        self.class_dataset=class_dataset
        self.vocab = vocab
        self.transform = transform

    def __getitem__(self, index):
        """Returns one data pair (image and caption)."""
        df = self.df
        vocab = self.vocab
        img_id=df.loc[index]
        image_path = self.root+img_id['image_file']
        caption=img_id['caption']
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        # Convert caption (string) to word ids.
        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        return image, target

    def __len__(self):
        return len(self.df)
    
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (image, caption).
    
    We should build custom collate_fn rather than using default collate_fn, 
    because merging caption (including padding) is not supported in default.

    Args:
        data: list of tuple (image, caption). 
            - image: torch tensor of shape (3, 256, 256).
            - caption: torch tensor of shape (?); variable length.

    Returns:
        images: torch tensor of shape (batch_size, 3, 256, 256).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded caption.
    """
    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions = zip(*data)

    # Merge images (from tuple of 3D tensor to 4D tensor).
    images = torch.stack(images, 0)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]        
    return images, targets, lengths

def idx2word(vocab, indices):
    sentence = []
    for i in range(params['batch_size']):
        indices[i].cpu().numpy()
    
    for index in indices:
        word = vocab.idx2word[index]
        sentence.append(word)
    return sentence

In [4]:
with open(params['vocab_path'], 'rb') as f:
    vocab = pickle.load(f)
transform = transforms.Compose([ 
        transforms.Resize(params['image_size']),
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])

train_dataset=CustomDataset(params['data_path'],params['train_csv'],'train',vocab,transform=transform)
val_dataset=CustomDataset(params['data_path'],params['val_csv'],'val',vocab,transform=transform)
train_dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)
val_dataloader=DataLoader(val_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)
encoder = EncoderCNN(params['embed_size']).to(device)
decoder = DecoderRNN(params['embed_size'], params['hidden_size'], len(vocab), params['num_layers']).to(device)
criterion = nn.CrossEntropyLoss()
model_param = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(model_param, lr=params['lr'], betas=(params['beta1'], params['beta2']))

/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
summary(decoder,[(params['batch_size'], params['embed_size']),(params['batch_size'], params['embed_size']),(64)])

TypeError: torch._VariableFunctionsClass.rand() argument after * must be an iterable, not int

In [ ]:
train=tqdm(train_dataloader)
count=0
train_loss = 0.0
for images,captions,lengths in train:
    images = images.to(device)
    captions = captions.to(device)
    targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
    features = encoder(images)
    outputs = decoder(features, captions, lengths)
    print(features.shape())
    print(captions.shape())
    print(lengths)

  0%|          | 0/6471 [00:01<?, ?it/s]


TypeError: 'torch.Size' object is not callable

In [ ]:
targets

tensor([   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    4,  146,    4,    4,    4,    4,    4,   33,
           4,   33,    4,    4,    4,    4,  170,    4,    4,    4,    4,    4,
           4,   81,    4,    4,    4,    4,    4,   51,    4,    4,  146,    4,
          33,   33,  146,    4,    4,    4,    4,  345,    4,    4,    4,    4,
           4,    4,  216,    4, 1361,    4,    4,    4,    4,    4,   48,    4,
           4,  194,    4,    4,    4,    4,    4,    4,   99,  131,  336,   13,
        1208,   92,  170,  908,  372, 2484, 7380,  113,   31, 5443,   78,  336,
          99, 3569,   99,   60,  331,   